### Example script for training MPNN-POM model

In [15]:
import deepchem as dc
import os
os.environ['TF_ENABLE_MLIR_OPTIMIZATIONS'] = '1'
from openpom.feat.graph_featurizer import GraphFeaturizer, GraphConvConstants
from openpom.utils.data_utils import get_class_imbalance_ratio
from openpom.models.mpnn_pom import MPNNPOMModel
from datetime import datetime
from constants import *

In [16]:
# TASKS = [
# 'alcoholic', 'aldehydic', 'alliaceous', 'almond', 'amber', 'animal',
# 'anisic', 'apple', 'apricot', 'aromatic', 'balsamic', 'banana', 'beefy',
# 'bergamot', 'berry', 'bitter', 'black currant', 'brandy', 'burnt',
# 'buttery', 'cabbage', 'camphoreous', 'caramellic', 'cedar', 'celery',
# 'chamomile', 'cheesy', 'cherry', 'chocolate', 'cinnamon', 'citrus', 'clean',
# 'clove', 'cocoa', 'coconut', 'coffee', 'cognac', 'cooked', 'cooling',
# 'cortex', 'coumarinic', 'creamy', 'cucumber', 'dairy', 'dry', 'earthy',
# 'ethereal', 'fatty', 'fermented', 'fishy', 'floral', 'fresh', 'fruit skin',
# 'fruity', 'garlic', 'gassy', 'geranium', 'grape', 'grapefruit', 'grassy',
# 'green', 'hawthorn', 'hay', 'hazelnut', 'herbal', 'honey', 'hyacinth',
# 'jasmin', 'juicy', 'ketonic', 'lactonic', 'lavender', 'leafy', 'leathery',
# 'lemon', 'lily', 'malty', 'meaty', 'medicinal', 'melon', 'metallic',
# 'milky', 'mint', 'muguet', 'mushroom', 'musk', 'musty', 'natural', 'nutty',
# 'odorless', 'oily', 'onion', 'orange', 'orangeflower', 'orris', 'ozone',
# 'peach', 'pear', 'phenolic', 'pine', 'pineapple', 'plum', 'popcorn',
# 'potato', 'powdery', 'pungent', 'radish', 'raspberry', 'ripe', 'roasted',
# 'rose', 'rummy', 'sandalwood', 'savory', 'sharp', 'smoky', 'soapy',
# 'solvent', 'sour', 'spicy', 'strawberry', 'sulfurous', 'sweaty', 'sweet',
# 'tea', 'terpenic', 'tobacco', 'tomato', 'tropical', 'vanilla', 'vegetable',
# 'vetiver', 'violet', 'warm', 'waxy', 'weedy', 'winey', 'woody'
# ]
print("No of tasks: ", len(gs_lf_tasks))

No of tasks:  138


In [19]:
# download curated dataset
# !wget https://raw.githubusercontent.com/ARY2260/openpom/main/openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv

# The curated dataset can also found at `openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv` in the repo.

input_file = '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/curated_GS_LF_merged_4983.csv' # or new downloaded file path

In [20]:
# get dataset
print(os.getcwd())
featurizer = GraphFeaturizer()
smiles_field = 'nonStereoSMILES'
loader = dc.data.CSVLoader(tasks=gs_lf_tasks,
                   feature_field=smiles_field,
                   featurizer=featurizer)
dataset = loader.create_dataset(inputs=[input_file])
n_tasks = len(dataset.tasks)

/Midgard/home/farzantn/phd/Olfaction/openpom


In [21]:
def pom_frame(pom_embeds, y, dir,required_desc ):
    # pom_embeds = model.predict_embedding(dataset)
    # y_preds = model.predict(dataset)
    # required_desc = list(dataset.tasks)
    type1 = {'floral': '#F3F1F7', 'subs': {'muguet': '#FAD7E6', 'lavender': '#8883BE', 'jasmin': '#BD81B7'}}
    type2 = {'meaty': '#F5EBE8', 'subs': {'savory': '#FBB360', 'beefy': '#7B382A', 'roasted': '#F7A69E'}}
    type3 = {'ethereal': '#F2F6EC', 'subs': {'cognac': '#BCE2D2', 'fermented': '#79944F', 'alcoholic': '#C2DA8F'}}
        
    # Assuming you have your features in the 'features' array
    pca = PCA(n_components=2, iterated_power=10)  # You can choose the number of components you want (e.g., 2 for 2D visualization)
    reduced_features = pca.fit_transform(pom_embeds) # try different variations

    variance_explained = pca.explained_variance_ratio_

    # Variance explained by PC1 and PC2
    variance_pc1 = variance_explained[0]
    variance_pc2 = variance_explained[1]

    # if is_preds:
    #     y = np.where(y_preds>threshold, 1.0, 0.0) # try quartile range (or rank)
    # else:
    #     y = dataset.y

    # Generate grid points to evaluate the KDE on (try kernel convolution)
    x_grid, y_grid = np.meshgrid(np.linspace(reduced_features[:, 0].min(), reduced_features[:, 0].max(), 500),
                                 np.linspace(reduced_features[:, 1].min(), reduced_features[:, 1].max(), 500))
    grid_points = np.vstack([x_grid.ravel(), y_grid.ravel()])
    def get_kde_values(label):
        plot_idx = required_desc.index(label)
        # print(y[:, plot_idx])
        label_indices = np.where(y[:, plot_idx] == 1)[0]
        kde_label = gaussian_kde(reduced_features[label_indices].T)
        kde_values_label = kde_label(grid_points)
        kde_values_label = kde_values_label.reshape(x_grid.shape)
        return kde_values_label
    
    def plot_contours(type_dictionary, bbox_to_anchor):
        main_label = list(type_dictionary.keys())[0]
        plt.contourf(x_grid, y_grid, get_kde_values(main_label), levels=1, colors=['#00000000',type_dictionary[main_label],type_dictionary[main_label]])
        legend_elements = []
        for label, color in type_dictionary['subs'].items():
            plt.contour(x_grid, y_grid, get_kde_values(label), levels=1, colors=color, linewidths=2)
            legend_elements.append(Patch(facecolor=color, label=label))
        legend = plt.legend(handles=legend_elements, title=main_label, bbox_to_anchor=bbox_to_anchor)
        legend.get_frame().set_facecolor(type_dictionary[main_label])
        plt.gca().add_artist(legend)

    plt.figure(figsize=(15, 10))
    plt.title('KDE Density Estimation with Contours in Reduced Space')
    plt.xlabel(f'Principal Component 1 ({round(variance_pc1*100, ndigits=2)}%)')
    plt.ylabel(f'Principal Component 2 ({round(variance_pc2*100, ndigits=2)}%)')
    plot_contours(type_dictionary=type1, bbox_to_anchor = (0.2, 0.8))
    plot_contours(type_dictionary=type2, bbox_to_anchor = (0.9, 0.4))
    plot_contours(type_dictionary=type3, bbox_to_anchor = (0.3, 0.1))
    # plt.colorbar(label='Density')
    # plt.show()
    # png_file = os.path.join(dir, 'pom_frame.png')
    # plt.savefig(png_file)
    plt.savefig("figs/realign_islands.png")
    plt.show()
    plt.close()


In [22]:
# get train valid test splits

randomstratifiedsplitter = dc.splits.RandomStratifiedSplitter()
train_dataset, test_dataset, valid_dataset = randomstratifiedsplitter.train_valid_test_split(dataset, frac_train = 0.8, frac_valid = 0.1, frac_test = 0.1, seed = 1)

In [23]:
print("train_dataset: ", len(train_dataset))
print("valid_dataset: ", len(valid_dataset))
print("test_dataset: ", len(test_dataset))


train_dataset:  3999
valid_dataset:  498
test_dataset:  486


In [24]:
train_ratios = get_class_imbalance_ratio(train_dataset)
assert len(train_ratios) == n_tasks

In [25]:
# learning_rate = ExponentialDecay(initial_rate=0.001, decay_rate=0.5, decay_steps=32*15, staircase=True)
learning_rate = 0.001

In [26]:
# initialize model
device_name = 'cuda'
model = MPNNPOMModel(n_tasks = n_tasks,
                            batch_size=128,
                            learning_rate=learning_rate,
                            class_imbalance_ratio = train_ratios,
                            loss_aggr_type = 'sum',
                            node_out_feats = 100,
                            edge_hidden_feats = 75,
                            edge_out_feats = 100,
                            num_step_message_passing = 5,
                            mpnn_residual = True,
                            message_aggregator_type = 'sum',
                            mode = 'classification',
                            number_atom_features = GraphConvConstants.ATOM_FDIM,
                            number_bond_features = GraphConvConstants.BOND_FDIM,
                            n_classes = 1,
                            readout_type = 'set2set',
                            num_step_set2set = 3,
                            num_layer_set2set = 2,
                            ffn_hidden_list= [392, 392],
                            ffn_embeddings = 256,
                            ffn_activation = 'relu',
                            ffn_dropout_p = 0.12,
                            ffn_dropout_at_input_no_act = False,
                            weight_decay = 1e-5,
                            self_loop = False,
                            optimizer_name = 'adam',
                            log_frequency = 32,
                            model_dir = './examples/experiments',
                            device_name=device_name)

In [27]:
nb_epoch = 200

In [28]:
metric = dc.metrics.Metric(dc.metrics.roc_auc_score)


In [29]:
train_dataset.w

array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       ...,
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]])

In [30]:
start_time = datetime.now()
for epoch in range(1, nb_epoch+1):
        loss = model.fit(
              train_dataset,
              nb_epoch=1,
              max_checkpoints_to_keep=1,
              deterministic=False,
              restore=epoch>1)
        train_scores = model.evaluate(train_dataset, [metric])['roc_auc_score']
        valid_scores = model.evaluate(valid_dataset, [metric])['roc_auc_score']
        print(f"epoch {epoch}/{nb_epoch} ; loss = {loss}; train_scores = {train_scores}; valid_scores = {valid_scores}")
model.save_checkpoint()
end_time = datetime.now()

torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138, 1]) torch.Size([128, 138])
torch.Size([128, 138

In [41]:
import pandas as pd
def prepare_mols_helper(input_file,tasks,mol_type="nonStereoSMILES"):
    featurizer = GraphFeaturizer()
    # smiles_field = 'nonStereoSMILES'
    loader = dc.data.CSVLoader(tasks=tasks,
                   feature_field=mol_type,
                   featurizer=featurizer
                          )
    dataset = loader.create_dataset(inputs=[input_file])
    df_mols = pd.read_csv(input_file)
    print(df_mols.columns)

    df_mols_embeddings_original=model.predict_embedding(dataset)
    print(df_mols_embeddings_original.shape)
        
    df_mols_embeddings=postproce_molembeddings(torch.from_numpy(df_mols_embeddings_original),df_mols['CID'])

    
    
     #z-score embeddings
    df_mols_embeddings_zscored = df_mols_embeddings.copy()
    scaled_features = StandardScaler().fit_transform(df_mols_embeddings_zscored.loc[:, 0:255].values.tolist())
    df_mols_embeddings_zscored.loc[:, 0:255] = pd.DataFrame(scaled_features, index=df_mols_embeddings_zscored.index, columns=df_mols_embeddings_zscored.columns[1:-1])
    df_mols_embeddings_zscored['Combined'] = df_mols_embeddings_zscored.loc[:, 0:255].values.tolist()
    
    
    
    
  
    return df_mols_embeddings_original,df_mols_embeddings,df_mols_embeddings_zscored

In [32]:
def postproce_molembeddings(embeddings,index):
    # molecules_embeddings_penultimate = torch.cat(embeddings)
    df_molecules_embeddings = pd.DataFrame(embeddings, index=index)
    df_molecules_embeddings['Combined'] = df_molecules_embeddings.loc[:, '0':'767'].values.tolist()
    df_molecules_embeddings=df_molecules_embeddings.reset_index()
    return(df_molecules_embeddings)

In [33]:
test_scores = model.evaluate(test_dataset, [metric])['roc_auc_score']
print("time_taken: ", str(end_time-start_time))
print("test_score: ", test_scores)

time_taken:  0:14:49.104568
test_score:  0.8397930849966729


In [42]:
import torch
from sklearn.preprocessing import StandardScaler

input_file_keller= '/local_storage/datasets/farzaneh/openpom/data/curated_datasets/curated_keller2016_nona.csv'
df_keller_temp=pd.read_csv(input_file_keller)
keller_tasks= df_keller_temp.columns.to_list()[5:]
df_mols_embeddings_original,df_mols_embeddings,df_mols_embeddings_zscored=prepare_mols_helper(input_file_keller,keller_tasks)

Index(['IsomericSMILES', 'nonStereoSMILES', 'CID', 'Subject', 'Replicate',
       'Acid', 'Ammonia', 'Bakery', 'Burnt', 'Chemical', 'Cold', 'Decayed',
       'Familiarity', 'Fish', 'Flower', 'Fruit', 'Garlic', 'Grass',
       'Intensity', 'Musky', 'Pleasantness', 'Sour', 'Spices', 'Sweaty',
       'Sweet', 'Warm', 'Wood'],
      dtype='object')
(15134, 256)


In [43]:
df_mols_embeddings_original

array([[-0.91714275, -0.5521736 ,  0.2802971 , ...,  0.6093874 ,
        -0.41447064,  0.01023541],
       [-0.91714275, -0.5521736 ,  0.2802971 , ...,  0.6093874 ,
        -0.41447064,  0.01023541],
       [-0.91714275, -0.5521736 ,  0.2802971 , ...,  0.6093874 ,
        -0.41447064,  0.01023541],
       ...,
       [-0.22834983, -0.5011051 ,  0.0310325 , ..., -0.09380654,
         0.6696509 ,  0.38331646],
       [-0.22834983, -0.5011051 ,  0.0310325 , ..., -0.09380654,
         0.6696509 ,  0.38331646],
       [-0.22834983, -0.5011051 ,  0.0310325 , ..., -0.09380654,
         0.6696509 ,  0.38331646]], dtype=float32)

In [44]:
df_mols_embeddings.to_csv("keller_pom_embeddings_14Apr.csv")


In [46]:
df_mols_embeddings_zscored.to_csv("keller_zscored_pom_embeddings_14Apr.csv")
